### KNN

En este notebook utilizaremos modelos de KNN para entrenar y clasificar el dataset con distintas configuraciones. Comenzaremos con la función de preprocesamiento que recrea lo realizado en la primera parte del TP, continuaremos con la función `preprocessing_significantes`, y finalizaremos preprocesando con las mejores variables seleccionadas con un árbol de decisión.

In [ ]:
# Modelo
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# Preprocessing
import preprocessing
import utils

# Búsqueda
from sklearn.model_selection import train_test_split, GridSearchCV

# Metrics
from sklearn.metrics import plot_roc_curve, roc_auc_score
from sklearn.metrics import classification_report, plot_confusion_matrix

# Plots
from matplotlib import pyplot as plt
import seaborn as sns

# Otros
import pandas as pd
import numpy as np

Definimos los hiperparámetros que probaremos y sus valores posibles, para la validación cruzada.

In [ ]:
params = [
    {'algorithm': ['ball_tree', 'kd_tree'],
         'n_neighbors': range(5, 150,5),
         'weights': ('uniform', 'distance'),
         'p': [1,2]},
    {'algorithm': ['brute'],
         'n_neighbors': range(5, 150, 10), 
         'weights': ('uniform', 'distance'),
         'p': [1,2]}
]
model = KNeighborsClassifier()

### Preprocessing base

Comenzamos por entrenar un modelo de KNN bajo las mismas condiciones en las que realizamos el baseline de la primera parte del TP, con la salvedad de que estandarizaremos las variables.

In [ ]:
df_base = utils.get_data()

In [ ]:
gscv_base = GridSearchCV(model, params, scoring='roc_auc', n_jobs=-1, cv=4, verbose=5)

In [ ]:
X = df_base.drop('tiene_alto_valor_adquisitivo',1)
Y = df_base['tiene_alto_valor_adquisitivo']

X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X,
                                                                        Y,
                                                                        stratify=Y,
                                                                        random_state=112)
X_train_base, X_test_base = preprocessing.standard_preprocessing_base_parte_1(X_train_base, X_test_base)

In [ ]:
%%time
gscv_base.fit(X_train_base, y_train_base)
score_base = roc_auc_score(y_test_base, gscv_base.predict_proba(X_test_base)[:,1])

In [ ]:
gscv_base.best_params_

In [ ]:
knn1 = gscv_base.best_estimator_
knn1.fit(X_train_base, y_train_base)

In [ ]:
score_base

Vemos un buen resultado en AUC-ROC, veamos cómo dieron las demás métricas

In [ ]:
print(classification_report(y_test_base,gscv_base.predict(X_test_base), digits=4))

En líneas generales son buenos resultados, con un accuracy equiparable al de la parte 1 de este TP. Lo que sigue siendo bastante pobre es la clasificación de casos de poder adquisitivo alto.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.grid(False)
plot_confusion_matrix(gscv_base, X_test_base, y_test_base, cmap=plt.cm.Blues, display_labels=['1', '0'], ax=ax)
plt.show()

In [ ]:
plot_roc_curve(gscv_base, X_test_base, y_test_base)

En general, podemos decir que es un buen clasificador. Veamos si podemos obtener mejores resultados trabajando de forma distinta con el preprocesamiento de los datos.

### Preprocessing Significantes

Repetimos el procedimiento anterior, con otro preprocesamiento.

In [ ]:
df = utils.get_data()

In [ ]:
params = [
    {'algorithm': ['ball_tree', 'kd_tree'],
         'n_neighbors': range(5, 150, 5),
         'weights': ('uniform', 'distance'),
         'p': [1,2]},
    {'algorithm': ['brute'],
         'n_neighbors': range(5, 150, 10), 
         'weights': ('uniform', 'distance'),
         'p': [1,2]}
]
model = KNeighborsClassifier()

In [ ]:
gscv = GridSearchCV(model, params, scoring='roc_auc', n_jobs=-1, cv=2, verbose=5)

In [ ]:
X = df[df.columns.drop('tiene_alto_valor_adquisitivo')]
Y = df['tiene_alto_valor_adquisitivo']
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, random_state=112)
X_train, X_test = preprocessing.preprocessing_significantes(X_train, X_test, 0.9)

In [ ]:
%%time
gscv.fit(X_train, y_train)
score = roc_auc_score(y_test, gscv.predict_proba(X_test)[:,1])

In [ ]:
gscv.best_params_

In [ ]:
knn2 = gscv.best_estimator_
knn2.fit(X_train, y_train)

In [ ]:
score

Hay un ligero empeoramiento en la métrica AUC-ROC

In [ ]:
print(classification_report(y_test,gscv.predict(X_test)))

Vemos que en el resto de las métricas los resultados tampoco mejoraron, y algunos de ellos empeoraron. 

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.grid(False)
plot_confusion_matrix(gscv, X_test, y_test, cmap=plt.cm.Blues, display_labels=['1', '0'], ax=ax)
plt.show()

In [ ]:
plot_roc_curve(gscv, X_test, y_test)

Vemos que los resultados, si bien no son pésimos, empeoraron respecto del primer caso. Continuemos con el último preprocesamiento que usaremos en este modelo.  

### Mejores variables seleccionadas con un arbol

En línea con lo analizado en el notebook de análisis de preprocessings, entrenaremos un modelo de KNN con las variables seleccionadas usando un árbol de decisión.

In [ ]:
df = utils.get_data()

In [ ]:
X = df[df.columns.drop('tiene_alto_valor_adquisitivo')]
X = pd.get_dummies(X)
Y = df['tiene_alto_valor_adquisitivo']

X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, random_state=112)

In [ ]:
X_train, X_test = preprocessing.standard_preprocessing_mejores_por_arbol(X_train, X_test)

In [ ]:
params = [
    {'algorithm': ['ball_tree', 'kd_tree'],
         'n_neighbors': range(5, 150, 5),
         'weights': ('uniform', 'distance'),
         'p': [1,2]},
    {'algorithm': ['brute'],
         'n_neighbors': range(5, 150, 10), 
         'weights': ('uniform', 'distance'),
         'p': [1,2]}
]

model = KNeighborsClassifier()

gscv1 = GridSearchCV(model, params, scoring='roc_auc', n_jobs=-1, cv=2, verbose = 4)

In [ ]:
%%time
gscv1.fit(X_train, y_train)

#### El mejor modelo:

In [ ]:
print(gscv1.best_estimator_)
print(gscv1.best_params_)
print(gscv1.best_score_)

Obtuvimos un modelo bastante similar al anterior, por lo que no sería esperable que haya una mejora considerable en las métricas que nos interesa analizar.

In [ ]:
knn3 = gscv1.best_estimator_
knn3.fit(X_train, y_train)

In [ ]:
print(classification_report(y_test,gscv1.predict(X_test),digits=4))

Efectivamente, obtuvimos resultados que no difieren en gran medida de lo obtenido anteriormente. 

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.grid(False)
plot_confusion_matrix(gscv1, X_test, y_test, cmap=plt.cm.Blues, display_labels=['1', '0'], ax=ax)
plt.show()

In [ ]:
plot_roc_curve(gscv1, X_test, y_test)

Vemos que, de los tres modelos entrenados, el último es el que mejores resultados tuvo, si bien esos resultados no distan en gran medida de los demás. Podemos atribuirle esta mejora en los resultados a la selección de variables realizada en el preprocesamiento.

# Persistiendo los modelos

In [ ]:
import pickle

In [ ]:
pickle.dump(knn1, open("Modelos/KNN/knn1.pickle", "wb"))
pickle.dump(knn2, open("Modelos/KNN/knn2.pickle", "wb"))
pickle.dump(knn3, open("Modelos/KNN/knn3.pickle", "wb"))

# Predicción de HoldOut

Usaremos para predecir el KNN obtenido para el preprocessing mejores variables según tree.

knn3 = pickle.load(open("Modelos/KNN/knn3.pickle", "rb"))

def holdout():
    dfv = utils.get_data()
    Xv = dfv[dfv.columns.drop('tiene_alto_valor_adquisitivo')]
    Xv = pd.get_dummies(Xv)
    

    ids, X = utils.get_holdout_data()
    X = pd.get_dummies(X)

    notInHoldout=[]
    for c in Xv.columns:
        if c not in X.columns:
            notInHoldout.append(c)
    X[notInHoldout] = 0

    Xv, X = preprocessing.standard_preprocessing_mejores_por_arbol(Xv, X)
    return ids, X

ids, X = holdout()

utils.escribir_holdout(knn3.predict(X), "2 - KNN", ids)